Import SysMLv2 Model:

In [1]:
import syside
import os

MAIN_DIR = "/Users/alejandronietocuatenta/Documents/sysmlv2_omg_introduction/"
# MODEL_DIR = "omg_single_example/"
# MODEL_DIR = "omg_introduction/" # mid complexity example
# MODEL_DIR = "omg_simple_vehicle/" # high complexity example
MODEL_DIR = "project_electronics_basics/"

# Construct full directory path
full_dir = os.path.join(MAIN_DIR, MODEL_DIR)

# Find the first file ending with .sysml
for file in os.listdir(full_dir):
    if file.endswith(".sysml"):
        MODEL_FILE_PATH = os.path.join(full_dir, file)
        break  # stop after finding the first .sysml file

(model, diagnostics) = syside.load_model([MODEL_FILE_PATH])

Check Model Class names:

In [2]:
for doc in model.user_docs:
    print(doc)
    with doc.lock() as locked:
        print(f"Model sexp: {syside.sexp(locked.root_node)}")

Model sexp: (Namespace
  (OwningMembership
    (Package model
      (OwningMembership
        (Package circuit_definitions
          (OwningMembership
            (PartDefinition Battery
              (Subclassification)
              (FeatureMembership
                (PortUsage +
                  (Subsetting)))
              (FeatureMembership
                (PortUsage -
                  (Subsetting)))))
          (OwningMembership
            (PartDefinition Resistor
              (Subclassification)
              (FeatureMembership
                (PortUsage a
                  (Subsetting)))
              (FeatureMembership
                (PortUsage b
                  (Subsetting)))))
          (OwningMembership
            (PartDefinition LED
              (Subclassification)
              (FeatureMembership
                (PortUsage a
                  (Subsetting)))
              (FeatureMembership
                (PortUsage k
                  (Subsetting)))
            

Check PartDefinition in Model:

In [3]:
for PartDefinition in model.nodes(syside.PartDefinition):
    print(PartDefinition)
    # print(f"\tis_individual: {partDefinition.is_individual}") # bool (not so relevant)
    print(f"\towned_ports: {len(list(PartDefinition.owned_ports))}")
    print(f"\towned_attributes: {len(list(PartDefinition.owned_attributes))}") # class LazyIterator

model::circuit_definitions::Battery
	owned_ports: 2
	owned_attributes: 0
model::circuit_definitions::Resistor
	owned_ports: 2
	owned_attributes: 0
model::circuit_definitions::LED
	owned_ports: 2
	owned_attributes: 3


Check PartUsage in Model:

In [4]:
for PartUsage in model.nodes(syside.PartUsage):
    print(PartUsage)
    # print(f"\tpart_definitions: {len(list(partUsage.part_definitions))}") # 0: Part, 1: Definition
    print(f"\tnested_parts: {len(list(PartUsage.nested_parts))}")
    print(f"\tnested_ports: {len(list(PartUsage.nested_ports))}")

model::circuit_connections::circuit
	nested_parts: 8
	nested_ports: 2
model::circuit_connections::b1
	nested_parts: 0
	nested_ports: 0
model::circuit_connections::circuit::r1
	nested_parts: 0
	nested_ports: 0
model::circuit_connections::circuit::r2
	nested_parts: 0
	nested_ports: 0
model::circuit_connections::circuit::led1
	nested_parts: 0
	nested_ports: 0


Evaluate Model attributes:

In [2]:
def safe(expr_func):
    try:
        return expr_func()
    except AttributeError:
        return None

for AttributeUsage in model.nodes(syside.AttributeUsage):
    print(AttributeUsage)
    # print(f"\tname: {attributeUsage.name}") # redunda info, already included in print(attributeUsage)
    print(f"\tfeature_value_expression: {safe(lambda: AttributeUsage.feature_value_expression.value)}") # need function "safe" defined in the above cell
    print(f"\tevaluate expression: {syside.Compiler().evaluate(AttributeUsage.feature_value_expression)[0]}")


model::circuit_definitions::Resistor::resistance
	feature_value_expression: None


TypeError: evaluate(): incompatible function arguments. The following argument types are supported:
    1. evaluate(self, expr: syside.core.Expression, stdlib: syside.core.Stdlib | None = None, scope: syside.core.Type | None | None = None) -> tuple[int | float | bool | syside.core.Infinity | str | range | syside.core.Element | syside.core.BoundMetaclass | None | list[int | float | bool | syside.core.Infinity | str | range | syside.core.Element | syside.core.BoundMetaclass | None] | None | None, syside.core.CompilationReport]

Invoked with types: syside.core.Compiler, NoneType

Check Constrain Arguments: 

In [3]:
for OperatorExpression in model.nodes(syside.OperatorExpression):
    # print(OperatorExpression)
    OperatorExpression.arguments.collect()